This is the analysis on brain connectivity data of 5 pairs of symmetrical nodes on certain freq. bands:
1) P3,P4,O1,O2 nodes for delta band 
2) C3,C4,F3,F4 nodes for theta band 
3) C3,C4,F3,F4 nodes for delta band 
4) C3,C4,T3,T4 nodes for theta band
5) C3,C4,P3,P4 nodes for theta band

In [28]:
import mne
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from Classification_models_LOSO_2Class import *
from model_functions import *

In [29]:
def remove_test(features,test):
    features_train = [features[i] for i in range(len(features)) if i not in test]
    #target_train = [targets[i] for i in range(len(targets)) if i not in test]
    return features_train
def remove_test_targets(targets, test):
    target_train = [targets[i] for i in range(len(targets)) if i not in test]
    return target_train

In [30]:
conn_data_1 = np.load('processed_data/con_data_01.npy',allow_pickle=True)
conn_data_2 = np.load('processed_data/con_data_02.npy',allow_pickle=True)
conn_data_3 = np.load('processed_data/con_data_03.npy',allow_pickle=True)
conn_data_4 = np.load('processed_data/con_data_04.npy',allow_pickle=True)
conn_data_5 = np.load('processed_data/con_data_05.npy',allow_pickle=True)

In [31]:
targets=[]
targets=[0 for i in range(26)]
targets.extend([2 for i in range(21)])
test= [49,53,63,71,72]

In [32]:
data_1 = remove_test(conn_data_1, test)
data_2 = remove_test(conn_data_2,test)
data_3 = remove_test(conn_data_3, test)
data_4 = remove_test(conn_data_4,test)
data_5 = remove_test(conn_data_5, test)

In [33]:
#targets= remove_test_targets(targetss, test)

In [34]:
targets= np.array(targets)
type(targets)

numpy.ndarray

In [35]:
len(targets)

47

In [36]:
data_1 = [data_1[i+36] for i in range(47)]
data_2 = [data_2[i+36] for i in range(47)]
data_3 = [data_3[i+36] for i in range(47)]
data_4 = [data_4[i+36] for i in range(47)]
data_5 = [data_5[i+36] for i in range(47)]

In [37]:
all_data = [data_1,data_2,data_3,data_4,data_5]

In [38]:
def get_arr(data, idx , epoch): 
    trials = [(data[idx][epoch].reshape(-1,))[i] for i in range(16) if (data[idx][epoch].reshape(-1,))[i] != 0]
    m = len(trials)
    return (np.array(trials)).reshape((1,m))


def get_all_freq_arr(all_data,idx, epoch):
    r = get_arr(all_data[0], idx , epoch)
    for i in range(1,len(all_data)):
        r = np.concatenate((r,get_arr(all_data[i], idx , epoch)),axis =1)
        
    #r=np.concatenate((get_arr(all_data[0], idx , epoch),get_arr(all_data[1], idx , epoch),get_arr(all_data[2], idx , epoch),get_arr(all_data[3], idx , epoch),get_arr(all_data[4], idx , epoch)), axis=1)
    return r
    

In [39]:
def get_all_epochs_data(all_data,idx):
    r = get_all_freq_arr(all_data,idx,0)
    for j in range(1,len(all_data[0][idx])):
        r = np.concatenate((r,get_all_freq_arr(all_data,idx,j)),axis = 0)
    return r

In [40]:
def get_all_features(all_data):
    output_array = [];
    for j in range(len(all_data[0])):
        r = get_all_epochs_data(all_data,j)
        output_array = output_array + [r]
    return output_array

In [41]:
features = get_all_features(all_data)

In [42]:
def train_prep(features,targets,exclude=None,flatten_final=True):
    total_subjects = len(targets)
    target_list = []
    for i in range(total_subjects):
        num_epochs = features[i].shape[0]
        target_list.append(targets[i]*np.ones(num_epochs))
    if exclude==None: 
        features_array = np.concatenate(features)
        targets_array = np.concatenate(target_list)
    else:
        features_array = np.concatenate(features[:exclude] + features[exclude+1:])
        targets_array  = np.concatenate(target_list[:exclude] + target_list[exclude+1:])
    if flatten_final:
        features_array = features_array.reshape((features_array.shape[0],-1))
    return features_array, targets_array

In [43]:
train_prep(features, targets,exclude=10, flatten_final=True)[0].shape

(656, 30)

In [44]:
def kNN_cross(rbps,targets,n_neighbors, PCA_components = 0):
   
    confusion_matrices_train = []
    confusion_matrices_test = []
    labels = np.unique(targets)
    for i in range(len(targets)):
        train_X, train_y = train_prep(rbps,targets,exclude=i,flatten_final=True)
        test_X = rbps[i].reshape(rbps[i].shape[0],-1)
        test_y = targets[i]*np.ones(rbps[i].shape[0])

        scaler = StandardScaler()
        train_X = scaler.fit_transform(train_X)
        test_X = scaler.transform(test_X)
        
        if PCA_components != 0:
            pca = PCA(n_components = PCA_components)
            train_X = pca.fit_transform(train_X, y = None)
            test_X = pca.transform(test_X)
        
        ThreeNN = KNeighborsClassifier(n_neighbors=n_neighbors)
        ThreeNN.fit(train_X, train_y)
        
        confusion_matrices_train += [confusion_matrix(train_y, ThreeNN.predict(train_X),labels=labels)]
        confusion_matrices_test += [confusion_matrix(test_y,ThreeNN.predict(test_X),labels=labels)]
    
    confusion_matrices_train = np.array(confusion_matrices_train)
    confusion_matrices_test = np.array(confusion_matrices_test)
    total_confusion_train = np.sum(confusion_matrices_train, axis= 0)
    total_confusion_test = np.sum(confusion_matrices_test, axis= 0)
    
    train_metrics_dict = {'acc':accuracy(total_confusion_train), 'sens':sensitivity(total_confusion_train), 
                            'spec':specificity(total_confusion_train), 'f1':f1(total_confusion_train)}
    test_metrics_dict = {'acc':accuracy(total_confusion_test), 'sens':sensitivity(total_confusion_test), 
                            'spec':specificity(total_confusion_test), 'f1':f1(total_confusion_test)}
    
    
    return train_metrics_dict, test_metrics_dict


In [45]:
kNN_cross(features, targets, n_neighbors=3, PCA_components = 0 )

({'acc': 1.0, 'sens': 1.0, 'spec': 1.0, 'f1': 1.0},
 {'acc': 0.5961251862891207,
  'sens': 0.5073529411764706,
  'spec': 0.656641604010025,
  'f1': 0.5045703839122485})

In [46]:
PCA_components = [5, 10, 20]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = kNN_cross(features, targets, n_neighbors=3, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [47]:
val_acc, train_acc

([0.6348733233979136, 0.6080476900149031, 0.6050670640834576],
 [0.9780664809175145, 0.9999028056761485, 1.0])

In [48]:
train_metric_dict, val_metric_dict = svm_cross(features, targets, kernel= "linear", reg_parameter = 1.0, degree=2, PCA_components= 0)
train_metric_dict, val_metric_dict

({'acc': 0.9237996501004342,
  'sens': 0.893781969309463,
  'spec': 0.9442628309905198,
  'f1': 0.9048466704425924},
 {'acc': 0.6810730253353204,
  'sens': 0.6580882352941176,
  'spec': 0.6967418546365914,
  'f1': 0.6258741258741258})

In [49]:
train_metric_dict, val_metric_dict = svm_cross(features, targets, kernel= "linear", reg_parameter = 10.0, degree=2, PCA_components= 0)
train_metric_dict, val_metric_dict

({'acc': 0.9350417935592561,
  'sens': 0.9211157289002557,
  'spec': 0.9445352511714068,
  'f1': 0.9199760526840949},
 {'acc': 0.706408345752608,
  'sens': 0.6875,
  'spec': 0.7192982456140351,
  'f1': 0.6549912434325744})

In [50]:
train_metric_dict, val_metric_dict = svm_cross(features, targets, kernel= "poly", reg_parameter = 1.0, degree=2, PCA_components= 0)
train_metric_dict, val_metric_dict

({'acc': 0.8245966435560164,
  'sens': 0.5791240409207161,
  'spec': 0.9919363626457448,
  'f1': 0.7280217019993971},
 {'acc': 0.6378539493293591,
  'sens': 0.2757352941176471,
  'spec': 0.8847117794486216,
  'f1': 0.3816793893129771})

In [51]:
train_metric_dict, val_metric_dict = log_reg_cross(features, targets)
train_metric_dict, val_metric_dict

({'acc': 0.9108728050281863,
  'sens': 0.8471067774936062,
  'spec': 0.9543423776833387,
  'f1': 0.8851309031692346},
 {'acc': 0.6929955290611028,
  'sens': 0.6838235294117647,
  'spec': 0.6992481203007519,
  'f1': 0.643598615916955})

In [52]:
train_metric_dict, val_metric_dict = log_reg_cross(features, targets, PCA_components = 15)
train_metric_dict, val_metric_dict

({'acc': 0.8663578047042053,
  'sens': 0.7803708439897699,
  'spec': 0.9249754821837202,
  'f1': 0.8256035175242041},
 {'acc': 0.7302533532041728,
  'sens': 0.6911764705882353,
  'spec': 0.7568922305764411,
  'f1': 0.6750448833034112})